In [1]:
import numpy as np
import cv2
from skimage.feature import hog
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [ ]:
from keras.datasets import cifar10
from skimage.feature import hog
import tensorflow as tf

# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Preprocess the dataset
# Reshape images to 1D array
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Step 2: Feature Extraction (HOG)
def extract_hog_features(images):
    hog_features = []
    for image in images:
        # Compute HOG features
        features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2-Hys")
        hog_features.append(features)
    return np.array(hog_features)

X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

# Step 3: Clustering (K-means)
def create_vocabulary(features, k):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(features)
    return kmeans.cluster_centers_

vocabulary = create_vocabulary(X_train_hog, k=100)  # Example: Use 100 clusters

# Step 4: Vector Quantization
def create_histograms(features, vocabulary):
    histograms = []
    for feature in features:
        distances = np.linalg.norm(vocabulary - feature.reshape(1, -1), axis=1)
        histogram = np.zeros(len(vocabulary))
        histogram[np.argmin(distances)] = 1
        histograms.append(histogram)
    return np.array(histograms)

X_train_histograms = create_histograms(X_train_hog, vocabulary)
X_test_histograms = create_histograms(X_test_hog, vocabulary)

# Step 5: Classification
classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))
classifier.fit(X_train_histograms, y_train)

# Step 6: Evaluation
y_pred = classifier.predict(X_test_histograms)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
